# Phase 2-2 : 小規模な知識グラフを作ってみる 【SpaCy】

このノートブックでは、spaCy を使って古典的NLPによるトリプルの抽出を試す。

## 目標

- 機械的なトリプル作成の基礎を理解する

## 参考

- [NetworkX Document](https://networkx.org/documentation/stable/reference/index.html)
- [spaCy](https://spacy.io/)

## ライブラリのインポート

In [ ]:
import spacy
from spacy.lang.en.examples import sentences

In [ ]:
nlp = spacy.load("en_core_web_sm")  # モデルのロード
doc = nlp(sentences[0])
print(doc.text)